<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [2]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 2.7 kB/s 


In [3]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

In [78]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy' -O input_seq.txt


--2021-11-14 11:07:12--  https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy
Resolving docs.google.com (docs.google.com)... 209.85.147.139, 209.85.147.100, 209.85.147.102, ...
Connecting to docs.google.com (docs.google.com)|209.85.147.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/eb25tooooqgbugk846276vt6s33f1opm/1636887975000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download [following]
--2021-11-14 11:07:12--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/eb25tooooqgbugk846276vt6s33f1opm/1636887975000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-10-4c-do

Major Triads, Minor Triads and Dom7 chords:

In [79]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-14 11:07:17--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 173.194.198.100, 173.194.198.139, 173.194.198.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.198.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/loqs56tsgd12f42huerbeduch44kk62n/1636887975000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-14 11:07:18--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/loqs56tsgd12f42huerbeduch44kk62n/1636887975000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4

In [278]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-14 13:28:29--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 142.250.136.113, 142.250.136.138, 142.250.136.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.136.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o8dnph7ig7rmi7c07ijbojqd67hcdacr/1636896450000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-14 13:28:29--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o8dnph7ig7rmi7c07ijbojqd67hcdacr/1636896450000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4

Meta Data:

In [279]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [280]:
data = [line.split(' ') for line in lines]

### Meta Data

In [281]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [282]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [283]:
assert len(data) == len(titles)
print(len(data))

1812


In [284]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [285]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [286]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [287]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [288]:
data_ngrams = list_to_ngrams(data, 3)

In [289]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [290]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3344
75,G7-C-C,1630
132,A7-Dm7-G7,1506
37,Em7-A7-Dm7,1132
410,C-C-C,1059
179,D7-G7-C,838
85,G7-G7-C,826
71,Am7-Dm7-G7,663
74,C-G7-C,638
6,Gm7-C7-F,615


In [291]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

# List Tunes for Test Data

In [292]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [293]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [294]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [295]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  data_tfidf = [" ".join(i) for i in data_ngrams]

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_model[n] = tfidf_vectorizer.fit_transform(data_tfidf)

In [296]:
tfidf_model

{1: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 2: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 3: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 4: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>}

## Test of the Contrafacts tunes

In [297]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034


# Doc2Vec Embeddings

## Read and Tag Data

In [298]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [299]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [300]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [301]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 #4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 1, 
                                 2, 
                                 #3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=20, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}, sample={sample}")
                        print(f"Corpus: {train_corpus[0]}")
                        model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                              dm = dm,
                                                              dm_concat = concat,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-14 13:28:41,270 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,mc2,t2)', 'datetime': '2021-11-14T13:28:41.270653', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-14 13:28:41,272 : INFO : collecting all words and their counts
2021-11-14 13:28:41,275 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-14 13:28:41,302 : INFO : collected 48 word types and 1812 unique tags from a corpus of 1812 examples and 82790 words
2021-11-14 13:28:41,304 : INFO : Creating a fresh vocabulary
2021-11-14 13:28:41,307 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 48 unique words (100.0%% of original 48, drops 0)', 'datetime': '2021-11-14T13:28:41.307135', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-14 13:28:41,624 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:41,632 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:41,634 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 283491 effective words/s
2021-11-14 13:28:41,884 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:41,896 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:41,898 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 327980 effective words/s
2021-11-14 13:28:42,175 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:42,188 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:42,190 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 297623 effective words/s
2021-11-14 13:28:42,447 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-14 13:28:47,265 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:47,281 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:47,283 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 320660 effective words/s
2021-11-14 13:28:47,550 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:47,562 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:47,564 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 315953 effective words/s
2021-11-14 13:28:47,828 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:28:47,843 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:28:47,845 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 311551 effective words/s
2021-11-14 13:28:48,108 : INFO : worker thread finished

...

2021-11-14 13:29:39,991 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:39,996 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:39,999 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.2s, 510334 effective words/s
2021-11-14 13:29:40,167 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:40,172 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:40,175 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.2s, 507049 effective words/s
2021-11-14 13:29:40,330 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:40,340 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:40,341 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.2s, 543237 effective words/s
2021-11-14 13:29:40,497 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=2, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-14 13:29:43,687 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:43,696 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:43,699 : INFO : EPOCH - 1 : training on 80978 raw words (82470 effective words) took 0.2s, 410360 effective words/s
2021-11-14 13:29:43,894 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:43,900 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:43,909 : INFO : EPOCH - 2 : training on 80978 raw words (82470 effective words) took 0.2s, 419088 effective words/s
2021-11-14 13:29:44,096 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:44,103 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:44,105 : INFO : EPOCH - 3 : training on 80978 raw words (82470 effective words) took 0.2s, 435475 effective words/s
2021-11-14 13:29:44,298 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=3, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-14 13:29:47,943 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:47,949 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:47,952 : INFO : EPOCH - 1 : training on 80978 raw words (82470 effective words) took 0.2s, 395737 effective words/s
2021-11-14 13:29:48,162 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:48,168 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:48,173 : INFO : EPOCH - 2 : training on 80978 raw words (82470 effective words) took 0.2s, 395139 effective words/s
2021-11-14 13:29:48,361 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 13:29:48,369 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 13:29:48,370 : INFO : EPOCH - 3 : training on 80978 raw words (82470 effective words) took 0.2s, 439279 effective words/s
2021-11-14 13:29:48,573 : INFO : worker thread finished

In [302]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-14 13:29:51,905 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model
2021-11-14 13:29:51,909 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.dv.* with mmap=None
2021-11-14 13:29:51,912 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.wv.* with mmap=None
2021-11-14 13:29:51,914 : INFO : setting ignored attribute cum_table to None
2021-11-14 13:29:51,917 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_1_3_0.1.model', 'datetime': '2021-11-14T13:29:51.917533', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-14 13:29:51,928 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model
2021-11-14 13:29:51,938 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model.dv.* with m

['autos_d2v_concat_1_3_0.1.model', 'autos_d2v_concat_2_2_0.1.model', 'autos_d2v_pv-dm_1_3_0.1.model', 'autos_d2v_pv-dbow_1_3_0.1.model', 'autos_d2v_concat_1_2_0.1.model', 'autos_d2v_pv-dm_1_2_0.1.model', 'autos_d2v_pv-dbow_2_2_0.1.model', 'autos_d2v_pv-dbow_1_2_0.1.model', 'autos_d2v_pv-dbow_2_3_0.1.model', 'autos_d2v_pv-dm_2_2_0.1.model', 'autos_d2v_concat_2_3_0.1.model', 'autos_d2v_pv-dm_2_3_0.1.model']
./models_doc2vec/autos_d2v_concat_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model


2021-11-14 13:29:52,132 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model', 'datetime': '2021-11-14T13:29:52.132193', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-14 13:29:52,135 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
2021-11-14 13:29:52,142 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model.dv.* with mmap=None
2021-11-14 13:29:52,144 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model.wv.* with mmap=None
2021-11-14 13:29:52,147 : INFO : setting ignored attribute cum_table to None
2021-11-14 13:29:52,171 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model', 'datetime': '2021-11-14T13:29:52.171027', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n

./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
./models_doc2vec/autos_d2v_concat_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model


In [303]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_concat_2_2_0.1.train', 'autos_d2v_pv-dm_1_3_0.1.train', 'autos_d2v_pv-dbow_1_2_0.1.train', 'autos_d2v_pv-dm_1_2_0.1.train', 'autos_d2v_pv-dbow_1_3_0.1.train', 'autos_d2v_concat_1_2_0.1.train', 'autos_d2v_concat_2_3_0.1.train', 'autos_d2v_pv-dm_2_3_0.1.train', 'autos_d2v_pv-dm_2_2_0.1.train', 'autos_d2v_concat_1_3_0.1.train', 'autos_d2v_pv-dbow_2_3_0.1.train', 'autos_d2v_pv-dbow_2_2_0.1.train']
./traindata_doc2vec/autos_d2v_concat_2_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_3_0.1.train
./traindata_doc2vec/autos_d2v_p

In [304]:
for name, m in models.items():
  print(m)

Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)


In [305]:
def compare_doc_models(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [306]:
# df_score = compare_doc_models(models)

In [307]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,14.459161
autos_d2v_concat_1_3_0.1.model,14.955850
autos_d2v_concat_1_4_0.1.model,14.679912
autos_d2v_concat_2_2_0.1.model,91.004415
autos_d2v_concat_2_3_0.1.model,91.059603
autos_d2v_concat_2_4_0.1.model,91.059603
autos_d2v_pv-dm_1_2_0.1.model,36.368653
autos_d2v_pv-dm_1_3_0.1.model,36.479029
autos_d2v_pv-dm_1_4_0.1.model,37.196468


In [308]:
def compare_doc_models2(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    topn = 5

    names = []
    scores = []
    runs = []
    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        tune_sim = 0

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=topn)
            if doc_id in [key for key, value in sims]:
              tune_sim += 1

        tunes_not_sim = len(train_corpus) - tune_sim
        score = 100 * tunes_not_sim/(tune_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in top {topn} place: {tune_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [309]:
df_score2 = compare_doc_models2(models)


*** Run 0
autos_d2v_concat_1_3_0.1.model
[TaggedDocument(words=['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], tags=[0]), TaggedDocument(words=['Am7', 'Am7', 'Cm7', 'Cm7', 'Eb', 'Eb', 'Em7', 'A7', 'Dm7', 'Dm7', 'Bm7', 'Bm7', 'Bbm7', 'Bbm7', 'Fm7', 'Fm7', 'E7', 'E7', 'Fm7', 'Fm7', 'C#', 'C#', 'Fm7', 'Fm7', 'C#', 'C#'], tags=[1]), TaggedDocument(words=['Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'E', 'E', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'Em7', 'Em7'], tags=[2]), TaggedDocument(words=['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C'

In [310]:
pd.pivot_table(df_score2, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,5.739514
autos_d2v_concat_1_3_0.1.model,5.629139
autos_d2v_concat_2_2_0.1.model,94.150110
autos_d2v_concat_2_3_0.1.model,93.543046
autos_d2v_pv-dbow_1_2_0.1.model,0.220751
autos_d2v_pv-dbow_1_3_0.1.model,0.220751
autos_d2v_pv-dbow_2_2_0.1.model,0.165563
autos_d2v_pv-dbow_2_3_0.1.model,0.165563
autos_d2v_pv-dm_1_2_0.1.model,22.185430


In [311]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211114_133129


In [312]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [313]:
 !zip -r models.zip models_doc2vec/ 

updating: models_doc2vec/ (stored 0%)
updating: models_doc2vec/autos_d2v_concat_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_2_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_concat_2_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model (deflated 9%)


In [314]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [315]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [316]:
#best_model = 'autos_d2v_pv-dbow_3_4_0.1'
best_model = 'autos_d2v_concat_1_3_0.1'

In [317]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [318]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [319]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1498, 1: 123, 2: 36, 3: 30, 4: 19, 5: 13, 6: 10, 8: 10, 7: 9, 11: 7, 15: 5, 9: 5, 77: 3, 19: 3, 12: 3, 10: 3, 14: 2, 29: 2, 23: 2, 34: 2, 30: 2, 20: 2, 293: 1, 46: 1, 33: 1, 74: 1, 91: 1, 27: 1, 111: 1, 80: 1, 133: 1, 31: 1, 36: 1, 32: 1, 107: 1, 98: 1, 21: 1, 38: 1, 18: 1, 372: 1, 16: 1, 85: 1, 26: 1, 35: 1, 101: 1})

Tunes similar to themselves in first place: 1498
Tunes not similar in first place: 314
% of tunes not similar to themselves in first place: 17.33%


In [320]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'A Certain Smile': 1
'A Foggy Day': 4
'A Sleepin' Bee': 3
'A Weaver Of Dreams': 7
'Ac-Cent-Tchu-Ate The Positive': 2
'After You': 11
'Afternoon In Paris': 1
'Alexander's Ragtime Band': 1
'All God's Chillun Got Rhythm': 1
'All Of A Sudden My Heart Sings': 2
'Always': 2
'Among My Souvenirs': 11
'Anthropology': 3
'April In Paris': 2
'As Long As I Live': 1
'At Last': 6
'At Long Last Love': 1
'Au Privave': 1
'Bags and Trane': 1
'Bags' Groove': 4
'Barbados': 5
'Barbara': 1
'Basin Street Blues': 1
'Baubles, Bangles and Beads': 2
'Bessie's Blues': 5
'Beyond The Sea': 2
'Bill Bailey': 8
'Billie's Bounce': 2
'Birth Of The Blues, The': 1
'Blackberry Winter': 1
'Blue Lou': 1
'Blue Monk': 4
'Blue Room, The': 1
'Blue Skies': 1
'Blue Sphere': 11
'Blue Turning Grey Over You': 15
'Blueberry Hill': 7
'Bluehawk': 2
'Blues For Alice': 11
'Blues In The Closet': 5
'Bluesette': 293
'Booker's Waltz': 46
'Bright Boy': 15
'But Beautifu

#### Determine Similarity of Tunes

In [321]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [322]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [323]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'Sweet Sue, Just You' is most similar to:
	0.856: Call Me
	0.836: Mack The Knife
	0.805: Smile
	0.763: Breeze And I, The
	0.756: My Secret Love
	0.750: Pent Up House
	0.738: When My Dreamboat Comes Home
	0.733: Three Little Words
	0.733: When You're Smilin'
	0.726: Ac-Cent-Tchu-Ate The Positive

'These Foolish Things' is most similar to:
	0.798: I Can't Get Started
	0.695: Cottontail
	0.672: I've Found A New Baby
	0.664: This Is All I Ask
	0.663: I Don't Want To Miss Mississippi
	0.662: All By Myself
	0.656: Summertime
	0.654: Could It Be You
	0.637: Doctor Jazz
	0.635: Put On A Happy Face

'Blue Moon' is most similar to:
	0.703: Daybreak
	0.673: Making Whoopee
	0.673: Flamingo
	0.632: African Queen, The
	0.585: Avalon
	0.579: There Will Never Be Another You
	0.572: Pagan Love Song
	0.553: When The Moon Comes Over The Mountain
	0.549: Love For Sale
	0.547: Impressions

'Hundred Years From Today, A' is most similar to:
	0.765: Hallelujah I Just Love Him So
	0.747: Harbour Lights
	0.725

#### Evaluate Similarity for Jazz Contrafacts



In [324]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,-0.287451
1,52nd Street Theme,I Got Rhythm,0.419318
2,Ablution,All The Things You Are,0.892721
3,Anthropology,I Got Rhythm,0.332569
4,Ballade,As Long As I Live,0.525637
5,Bright Mississippi,Sweet Georgia Brown,0.777302
6,C.T.A.,I Got Rhythm,0.243116
7,Celia,I Got Rhythm,-0.100487
8,Cottontail,I Got Rhythm,0.626069
9,Countdown,Tune Up,0.284189


##### Compare results from TF-IDF and doc2vec



In [325]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989,-0.287451
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865,0.419318
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596,0.892721
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179,0.332569
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000,0.525637
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051,0.777302
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436,0.243116
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649,-0.100487
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247,0.626069
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034,0.284189


##### Verbose Testing Output

In [326]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': -0.2874508798122406
'26-2' is most similar to:
	0.701: 500 Miles High
	0.686: St. Thomas
	0.682: Perhaps
	0.675: He's A Tramp
	0.662: Estate

'52nd Street Theme' to 'I Got Rhythm': 0.41931816935539246
'52nd Street Theme' is most similar to:
	0.896: I've Got My Fingers Crossed
	0.895: Affirmation
	0.833: Little Sunflower
	0.823: Why Do I Love You
	0.823: Shutterbug

'Ablution' to 'All The Things You Are': 0.8927210569381714
'Ablution' is most similar to:
	0.893: All The Things You Are
	0.779: Jinrikisha
	0.777: Sippin' At Bells
	0.753: On The Sunny Side Of The Street
	0.739: A Lovely Way To Spend An Evening

'Anthropology' to 'I Got Rhythm': 0.33256906270980835
'Anthropology' is most similar to:
	0.943: Salt Peanuts
	0.943: Passport
	0.935: Moose The Mooche
	0.924: Theme, The
	0.913: No Moe

'Ballade' to 'As Long As I Live': 0.525637149810791
'Ballade' is most similar to:
	0.814: Agua De Beber
	0.781: With A Song In My Heart
	0.774: Tickle-Toe
	0.734: On Green

In [327]:
model.dv.similarity(0, 1)

0.7010187

#### Download Model

In [276]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>